In [1]:
import os

import utils

In [2]:
%%time

# Set up CaseClass object
casename = "g.e22.G1850ECO_JRA_HR.TL319_t13.004"

# Directories to search for netCDF files
run_root = os.path.join(os.sep, "glade", "scratch", "mlevy")
archive_hist_root = os.path.join(os.sep, "glade", "scratch", "mlevy", "archive")
campaign_root = os.path.join(os.sep, "glade", "campaign", "cesm", "development", "bgcwg", "projects", "hi-res_JRA", "cases")

utils.timeseries_and_history_comparison(casename, run_root, archive_hist_root, campaign_root)

Checking year 0001...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h ...
All variables available in time series for year 0001
----
Checking year 0002...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h1 ...
... checking stream cice.h ...
All variables available in time series for year 0002
----
Checking year 0003...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h1 ...
... checking stream cice.h ...
All variables available in time series for year 0003
----
Checking year 0004...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h1 ...
... checking stream cice.h ...
All variables available in time series for year 0004
----
Checking year 0005...
... checking stream pop.h.